In [1]:
VERSION

v"1.12.0"

## Redefinition of constants (structs)

https://julialang.org/blog/2025/10/julia-1.12-highlights/#redefinition_of_constants_structs

Bindings now participate in the "world age" mechanism previously used for methods. This has the effect that constants and structs can be properly redefined. As an example:

In [2]:
struct Foo{T} a::T end
f(foo::Foo, x) = ((; a) = foo; a*x)
@show foo = Foo(3)
f(foo, 4)

foo = Foo(3) = Foo{Int64}(3)


12

In [3]:
struct Foo{T} a::T; b::T end
@show foo = Foo(3, 4)
f(foo, 4)

foo = Foo(3, 4) = Foo{Int64}(3, 4)


LoadError: MethodError: no method matching f(::Foo{Int64}, ::Int64)
The function `f` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  f([91m::@world(Foo, 38668:38673)[39m, ::Any)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[2]:2[24m[39m


In [4]:
f(foo::Foo, x) = ((; a, b) = foo; a*x + b)
f(foo, 4)

16

## Score test of risk ratio for 2x2 tables

In [5]:
using Distributions
using StatsFuns
using Roots

safemul(x, y) = x == 0 ? zero(x*y) : x*y
safediv(x, y) = x == 0 ? zero(x/y) : x/y

_riskratiohat(a, b, c, d) = safediv(a*(c+d), (a+b)*c)

function Delta(a, b, c, d; ρ=1.0)
    m, n = a+b, c+d
    A, B, C = ρ-1, n-a+ρ*(m-c), a*n-ρ*m*c
    Δ = isinf(ρ) ? oftype(ρ, -c) : ρ==0 ? oftype(ρ, a) : safediv(2C, B + √(B^2 - 4A*C))
end

function _chisqstat_rr(a, b, c, d, Δ)
    m, n = a+b, c+d
    safemul(Δ^2, safediv(b, m*(a-Δ)) + safediv(d, n*(c+Δ)))
end

function chisqstat_rr(a, b, c, d; ρ=1.0)
    Δ = Delta(a, b, c, d; ρ)
    _chisqstat_rr(a, b, c, d, Δ)
end

function pvalue_rr_score(a, b, c, d; ρ=1.0)
    χ² = chisqstat_rr(a, b, c, d; ρ)
    ccdf(Chisq(1), χ²)
end

function confint_rr_score(a, b, c, d; α=0.05)
    (a+b==0 || c+d==0 || a+c==0 || b+d==0) && return [0.0, Inf]
    f(logρ) = logit(pvalue_rr_score(a, b, c, d; ρ=exp(logρ))) - logit(α)
    L = if f(-Inf) > 0
        -Inf
    else
        logRRhat = log(_riskratiohat(a, b, c, d))
        x0 = logRRhat == -Inf ? -10.0 : logRRhat == Inf ? 10.0 : logRRhat - 1
        find_zero(f, x0)
    end
    U = if f(Inf) > 0
        Inf
    else
        logRRhat = log(_riskratiohat(a, b, c, d))
        x0 = logRRhat == -Inf ? -10.0 : logRRhat == Inf ? 10.0 : logRRhat + 1
        find_zero(f, x0)
    end
    exp(L), exp(U)
end

a, b, c, d = 35, 15, 25, 25
ρ = 1
α = 0.05
@show [a b; c d]
@show ρ
@show α
@show pvalue_rr_score(a, b, c, d; ρ=1)
@show _riskratiohat(a, b, c, d)
@show confint_rr_score(a, b, c, d);

[a b; c d] = [35 15; 25 25]
ρ = 1
α = 0.05
pvalue_rr_score(a, b, c, d; ρ = 1) = 0.041226833337163676
_riskratiohat(a, b, c, d) = 1.4
confint_rr_score(a, b, c, d) = (1.013489079101118, 1.9850551032224117)


In [6]:
struct ScoreTest2x2RR{I, R}
    a::I
    b::I
    c::I
    d::I
    ρ::R
    α::R
    pvalue::R
    RRhat::R
    CI::Tuple{R, R}
end

ScoreTest2x2RR(a, b, c, d; ρ=1.0, α=0.05) =
    ScoreTest2x2RR(a, b, c, d, float(ρ), float(α),
        pvalue_rr_score(a, b, c, d; ρ=float(ρ)),
        _riskratiohat(a, b, c, d),
        confint_rr_score(a, b, c, d; α=float(α))
    )

a, b, c, d = 35, 15, 30, 30
ρ = 1
α = 0.05
result = ScoreTest2x2RR(a, b, c, d; ρ, α)

ScoreTest2x2RR{Int64, Float64}(35, 15, 30, 30, 1.0, 0.05, 0.033641721711823565, 1.4, (1.0268842266244511, 1.9336861464533983))

In [7]:
@show result.a
@show result.b
@show result.c
@show result.d
@show result.ρ
@show result.α
@show result.pvalue
@show result.RRhat
@show result.CI; 

result.a = 35
result.b = 15
result.c = 30
result.d = 30
result.ρ = 1.0
result.α = 0.05
result.pvalue = 0.033641721711823565
result.RRhat = 1.4
result.CI = (1.0268842266244511, 1.9336861464533983)


In [8]:
function Base.show(io::IO, ::MIME"text/plain", x::ScoreTest2x2RR)
    r(x) = round(x; sigdigits=3)
    (; a, b, c, d, ρ, α, pvalue, RRhat, CI) = x
    print(io, "Pearson's chi-squared test for risk ratio\n")
    print(io, "  data: ", [a b; c d], "\n")
    print(io, "  testing risk ratio: ", ρ, "\n")
    print(io, "  P-value: ", r(100pvalue), " %\n")
    print(io, "  point estimate of risk ratio: ", r(RRhat), "\n")
    print(io, "  $(100(1 - α)) % confidence interval of risk ratio: [", r(CI[1]), ", ", r(CI[2]), "]\n")
end

a, b, c, d = 35, 15, 30, 30
ρ = 1
α = 0.05
ScoreTest2x2RR(a, b, c, d; ρ, α)

Pearson's chi-squared test for risk ratio
  data: [35 15; 30 30]
  testing risk ratio: 1.0
  P-value: 3.36 %
  point estimate of risk ratio: 1.4
  95.0 % confidence interval of risk ratio: [1.03, 1.93]


In [9]:
function Base.show(io::IO, x::ScoreTest2x2RR)
    r(x) = round(x; sigdigits=3)
    (; a, b, c, d, ρ, α, pvalue, RRhat, CI) = x
    print(io,
        "ScoreTest2x2RR(",
        "a=", a, ", ",
        "b=", b, ", ",
        "c=", c, ", ",
        "d=", d, ", ",
        "ρ=", r(ρ), ", ",
        "α=", r(α), ", ",
        "pvalue=", r(pvalue), ", ",
        "RRhat=", r(RRhat), ", ",
        "CI=", r.(CI), ")")
end

a, b, c, d = 35, 15, 30, 30
ρ = 1
α = 0.05
show(ScoreTest2x2RR(a, b, c, d; ρ, α));

ScoreTest2x2RR(a=35, b=15, c=30, d=30, ρ=1.0, α=0.05, pvalue=0.0336, RRhat=1.4, CI=(1.03, 1.93))